### Testing of the code during development

Looking at the M1-1 dataset

In [3]:
import os
import glob
from oyv.imports import *
import importlib
import warnings

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


### Find modules and  files

In [19]:
# Load the modules
os.chdir('/home/oyvindl/work/code/git_public/sigpyproc/examples/')
os.chdir('../')
from sigpyproc import sig_load, sig_append, sig_calc, sig_draft, sig_open_water_correction
os.chdir('examples/')

In [27]:
# Load converted data from M1-1
floc = '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/*'
flist = glob.glob(floc)
for fn in flist:
    if '_avgd' in fn:
        flist.remove(fn)

In [28]:
flist

['/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_7.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_1.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_2.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_3.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_4.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_5.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_6.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_7.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_AeN_M1_01_8.mat',
 '/media/oyvindl/ratatosk/data/mooring_data/signature500/m1_1/converted/S100812A002_A

### Make Dataset

In [6]:
DX = sig_load.matfiles_to_dataset(flist)

/home/oyvindl/work/code/git_public/sigpyproc/sigpyproc/sig_append.py:274: RuntimeWarning: invalid value encountered in sqrt
  180 / np.pi* np.arccos(np.sqrt(
/home/oyvindl/work/code/git_public/sigpyproc/sigpyproc/sig_append.py:283: RuntimeWarning: invalid value encountered in sqrt
  180 / np.pi* np.arccos(np.sqrt(


1870992 time points, 38979 ensembles. Sample per ensemble: 48
Done. Run sig_funcs.overview() to print some additional details.


In [30]:
DX.SEA_ICE_DRAFT_AST

<xarray.DataArray 'SEA_ICE_DRAFT_AST' (TIME: 38979, SAMPLE: 48)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * TIME     (TIME) float64 1.781e+04 1.781e+04 ... 1.822e+04 1.822e+04
  * SAMPLE   (SAMPLE) int64 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48
Attributes:
    long_name:  Sea ice draft at each sample (AST)
    units:      m
    note:       From AST altimeter distances.\n\nComputed with the function s...

In [31]:
sig_load.overview(DX)


TIME RANGE:
06 Oct 2018 12:00  -->  16 Nov 2019 12:30  (406.0 days)
Time between ensembles: 15.0 min.
Time between samples in ensembles: 1.0 sec.

PRESSURE:
Median (STD) of altimeter pressure: 20.6 dbar (0.5 dbar)  - with fixed atm offset 9.500 dbar.

SIZE:
Total 1870992 time points.
Split into 38979 ensembles with 48 sample per ensemble.
Ocean velocity bins: 16.


#### Append AMSR2 SIC

In [32]:
sic = xr.open_dataset('/home/oyvindl/work/projects/sig500/data/misc/sic/M1M2_AMSR2_SIC.nc')

In [33]:
DX = sig_append.add_to_sigdata(DX, sic.SIC_m1, sic.time+0.5, 'SIC_AMSR2', 
                               attrs = {'sourcefile':'/home/oyvindl/work/projects/sig500/data/misc/sic/M1M2_AMSR2_SIC.nc'})

In [11]:
fig, ax = plt.subplots(figsize = (12, 4))
ax.plot(DX.TIME, DX.SIC_AMSR2, 'k', zorder = 20, lw = 1, label ='AMSR2 SIC (nearest cell)')
ax.plot(DX.TIME, DX.SIC_FOM, color = col.col('o'), alpha = 0.1)
SIC_FOM_dat, td = stats.dmean(DX.SIC_FOM, DX.TIME)
ax.plot(td+0.5, SIC_FOM_dat, color = col.col('o'), alpha = 1, label = 'SIC_FOM (daily)')
SIC_FOM_ALT_dat, td = stats.dmean(DX.SIC_FOM_ALT, DX.TIME)
ax.plot(td+0.5, SIC_FOM_ALT_dat, color = col.col('g'), alpha = 1, lw = 1, label = 'SIC_FOM_ALT (daily)')
ax.legend()
SIC_AMSR2_dat, td = stats.dmean(DX.SIC_AMSR2, DX.TIME)

pl.md_xlab(ax)

In [34]:
pearsonr(SIC_AMSR2_dat, SIC_FOM_ALT_dat)

NameError: name 'pearsonr' is not defined

In [ ]:
pearsonr(SIC_AMSR2_dat, SIC_FOM_dat)

#### Append msl and CTD


In [35]:
dmsl = pickle_load('../testing/development/CARRA_MSL_M1_temp.p')

In [36]:
DX = sig_append.append_atm_pres(DX, dmsl.MSL, dmsl.time)

In [14]:
dctd = pickle_load('../testing/development/CARRA_CTD_M1_1_temp.p')

In [15]:
DX = sig_append.set_lon(DX, 28.10)
DX = sig_append.set_lat(DX, 79.59)

In [16]:
DX = sig_append.append_ctd(DX, dctd.TEMP, dctd.SP, dctd.pres, dctd.time)

In [17]:
# Calculate depth
DX = sig_calc.dep_from_p(DX, corr_CTD_density=True)

# Draft

In [18]:
DX = sig_draft.calculate_draft(DX)

/home/oyvindl/anaconda3/envs/oyv/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


### Run BETA finder

In [131]:
DX = sig_open_water_correction.get_Beta_from_OWSD(DX)

In [132]:
sig_open_water_correction.compare_OW_correction(DX)

/home/oyvindl/anaconda3/envs/oyv/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


LE: Mean (median) offset: 17.7 cm (18.2 cm)
AST: Mean (median) offset: 4.0 cm (3.8 cm)
LE: Mean (median) dBETA: 9.0 (9.3)
AST: Mean (median) dBETA: 2.0 (1.9)
LE - MEAN SEA ICE DRAFT:
Before OW correction: 1.41 m
After OW correction: 1.27 m
AST - MEAN SEA ICE DRAFT:
Before OW correction: 1.15 m
After OW correction: 1.11 m


### Recompute draft

In [ ]:
DX = sig_draft.calculate_draft(DX)